In [1]:
import matplotlib.pyplot as plt
import numpy as np

from qpspin_mc.two_level_system import SimulationParameters, SystemParameters, TwoLevelSystemSimulator

In [2]:
gammas = np.linspace(0.001, 5, 10)
beta = 1
h = 1

SEED = 551818
N_STEPS = 100_000
N_SAMPLES = 1_000
N_INIT_STEPS = 1_000

In [3]:
m_estimates = []
m_errs = []
acc_rates = []

for i, g in enumerate(gammas):
    system_parameters = SystemParameters(beta=beta, h=h, gamma=g)

    simulation_parameters = SimulationParameters(
        n_steps=N_STEPS, n_samples=N_SAMPLES, n_init_steps=N_INIT_STEPS, seed=[i, SEED]
    )

    mc_result = TwoLevelSystemSimulator(system_parameters=system_parameters).run_mc(
        simulation_parameters
    )
    m_hat, m_sem = mc_result.estimate_magnetization()
    m_estimates.append(m_hat)
    m_errs.append(m_sem)
    acc_rates.append(mc_result.acceptance_rate())

m_estimates = np.array(m_estimates)
m_errs = np.array(m_errs)
acc_rates = np.array(acc_rates)

In [ ]:
gs = np.linspace(gammas.min(), gammas.max(), 100)
bs = np.sqrt(h**2 + gs**2)
m_exact = beta * gs**2 / bs**2  * np.tanh(beta * bs)


fig, ax = plt.subplots()
ax.plot(gs, m_exact, label="Exact")
ax.errorbar(gammas, m_estimates, yerr=m_errs, fmt=".", label="MC estimate")
ax.plot(gammas, acc_rates, "kx", label="Acceptance rate")
ax.legend()